# 第 5 章: 係り受け解析


### 40. 係り受け解析結果の読み込み（形態素)

### 41. 係り受け解析結果の読み込み（文節・係り受け）


In [1]:
from pathlib import Path
from collections import defaultdict

class Morph:
    def __init__(self, surface: str, base: str, pos: str, pos1: str):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

class Chunk:
    def __init__(self, morphs: list[Morph], dst: int | None, srcs: list[int]):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs

    def to_str(self) -> str:
        text = "".join([morph.surface for morph in self.morphs if morph.pos != "記号"])
        return text

class CabochaParser():
    def __init__(self):
        self.article = [] # list of sentence
        self.sentence = [] # list of chunk
        self.kakari_dict = defaultdict(lambda: [])
        self.morphs = []
        self.dst = None

    def parse_line(self, line: str):
        # 表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音
        # -> 表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）

        if line.strip() == "":
            return

        # 係り結びを解析する処理
        if line.strip()[0] == "*":
            if len(self.morphs) > 0:
                # 今のチャンクを処理
                assert self.dst != None
                chunk = Chunk(
                    morphs=self.morphs,
                    dst=self.dst if self.dst != -1 else None,
                    srcs=[]
                )
                self.sentence.append(chunk)
                self.morphs = []
                self.dst = None

            line = line.strip()
            splitted = line.split(" ")
            src = int(splitted[1])
            self.dst = int(splitted[2][:-1])
            self.kakari_dict[self.dst].append(src)
            return

        if line.strip() == "EOS":
            if len(self.morphs) == 0:
                # EOSが2回続いたとき
                return
            # 今のチャンクを処理
            assert self.dst != None
            chunk = Chunk(
                morphs=self.morphs,
                dst=self.dst if self.dst != -1 else None,
                srcs=[]
            )
            self.sentence.append(chunk)
            self.morphs = []
            self.dst = None

            # EOSが来たら係り結びの対応を更新
            for i in range(len(self.sentence)):
                if self.kakari_dict[i] != []:
                    self.sentence[i].srcs = self.kakari_dict[i]
            self.kakari_dict = defaultdict(lambda: [])

            # 今の文の処理
            self.article.append(self.sentence)
            self.sentence = []
            return

        # 形態素を解析する処理
        surface, rest = line.split("\t")
        outputs = rest.split(",")
        morph = Morph(
            surface=surface,
            base=outputs[-3],
            pos=outputs[0],
            pos1=outputs[1]
        )
        self.morphs.append(morph)

def read_cabocha(filepath: str | Path) -> list[list[Chunk]]:
    parser = CabochaParser()
    with open(filepath) as f:
        for line in f:
            parser.parse_line(line)
    return parser.article

article = read_cabocha("./data/ai.ja.txt.parsed")

In [2]:
for chunk in (sentence := article[1]):
    if chunk.dst is None:
        continue
    print("文節  : ", end="")
    for morph in chunk.morphs:
        print(morph.surface, end="")
    print("\n係り先: ", end="")
    for dst_morph in sentence[chunk.dst].morphs:
        print(dst_morph.surface, end="")

文節  : 人工知能
係り先: 語。文節  : （じんこうちのう、、
係り先: 語。文節  : AI
係り先: 〈エーアイ〉）とは、文節  : 〈エーアイ〉）とは、
係り先: 語。文節  : 「『計算
係り先: （）』という文節  : （）』という
係り先: 道具を文節  : 概念と
係り先: 道具を文節  : 『コンピュータ
係り先: （）』という文節  : （）』という
係り先: 道具を文節  : 道具を
係り先: 用いて文節  : 用いて
係り先: 研究する文節  : 『知能』を
係り先: 研究する文節  : 研究する
係り先: 計算機科学文節  : 計算機科学
係り先: （）の文節  : （）の
係り先: 一分野」を文節  : 一分野」を
係り先: 指す文節  : 指す
係り先: 語。文節  : 語。
係り先: 研究分野」とも文節  : 「言語の
係り先: 推論、文節  : 理解や
係り先: 推論、文節  : 推論、
係り先: 問題解決などの文節  : 問題解決などの
係り先: 知的行動を文節  : 知的行動を
係り先: 代わって文節  : 人間に
係り先: 代わって文節  : 代わって
係り先: 行わせる文節  : コンピューターに
係り先: 行わせる文節  : 行わせる
係り先: 技術」、または、文節  : 技術」、または、
係り先: 研究分野」とも文節  : 「計算機
係り先: （コンピュータ）による文節  : （コンピュータ）による
係り先: 情報処理システムの文節  : 知的な
係り先: 情報処理システムの文節  : 情報処理システムの
係り先: 実現に関する文節  : 設計や
係り先: 実現に関する文節  : 実現に関する
係り先: 研究分野」とも文節  : 研究分野」とも
係り先: される。

### 42. 係り元と係り先の文節の表示


In [3]:
for sentence in article[:3]:
    for chunk in sentence:
        if chunk.dst is None:
            continue
        src = chunk.to_str()
        dst = sentence[chunk.dst].to_str()
        print(f"{src}\t{dst}")

人工知能	語
じんこうちのう	語
AI	エーアイとは
エーアイとは	語
計算	という
という	道具を
概念と	道具を
コンピュータ	という
という	道具を
道具を	用いて
用いて	研究する
知能を	研究する
研究する	計算機科学
計算機科学	の
の	一分野を
一分野を	指す
指す	語
語	研究分野とも
言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
代わって	行わせる
コンピューターに	行わせる
行わせる	技術または
技術または	研究分野とも
計算機	コンピュータによる
コンピュータによる	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野とも
研究分野とも	される
日本大百科全書(ニッポニカ)』の	解説で
解説で	述べている
情報工学者通信工学者の	佐藤理史は
佐藤理史は	述べている
次のように	述べている


### 43. 名詞を含む文節が動詞を含む文節に係るものを抽出


In [4]:
for sentence in article[:3]:
    for chunk in sentence:
        if chunk.dst is None:
            continue
        if "名詞" not in [morph.pos for morph in chunk.morphs]:
            continue
        dst_chunk = sentence[chunk.dst]
        if "動詞" not in [morph.pos for morph in dst_chunk.morphs]:
            continue

        src = chunk.to_str()
        dst = dst_chunk.to_str()

        print(f"{src}\t{dst}")

道具を	用いて
知能を	研究する
一分野を	指す
知的行動を	代わって
人間に	代わって
コンピューターに	行わせる
研究分野とも	される
解説で	述べている
佐藤理史は	述べている
次のように	述べている


### 44. 係り受け木の可視化


In [5]:
import graphviz as gv

def visualize_kakariuke(sentence: list[Chunk]):
    graph = gv.Digraph(format="png", filename="test")
    for chunk in sentence:
        if chunk.dst is None:
            continue
        dst_chunk = sentence[chunk.dst]
        graph.node(chunk.to_str())
        graph.edge(chunk.to_str(), dst_chunk.to_str())
    graph.render()

ex_article = read_cabocha("./data/ch5_example.parsed")
# ex_article = read_cabocha("./data/ch5-47.parsed")
visualize_kakariuke(ex_article[0])

### 45. 動詞の格パターンの抽出


In [6]:
lines = []
for sentence in article:
    for chunk in sentence:
        try:
            i_verb = [morph.pos for morph in chunk.morphs].index("動詞")
            dst_morph = chunk.morphs[i_verb]
        except:
            continue

        src_morphs = []
        for src in chunk.srcs:
            try:
                src_chunk = sentence[src]
                i_joshi = [morph.pos for morph in src_chunk.morphs][::-1].index("助詞")
                src_morph = src_chunk.morphs[-(i_joshi + 1)]
                src_morphs.append(src_morph)
            except:
                continue

        if len(src_morphs) == 0:
            continue

        lines.append(f"{dst_morph.base}\t{' '.join(morph.base for morph in src_morphs)}")
with open("./data/output-ch5-45.txt", "w") as f:
    f.writelines(line + "\n" for line in lines)

In [7]:
# l = [1, 2, 3, 5, 6, 7, 8]
# i = l[::-1].index(7)
# # print(i)
# l[-(i+1)]

# !sort ./data/output-ch5-45.txt
# !sort ./data/output-ch5-45.txt | grep -E "行う|なる|与える"

### 46. 動詞の格フレーム情報の抽出


In [8]:
for sentence in article[:5]:
    for chunk in sentence:
        try:
            i_verb = [morph.pos for morph in chunk.morphs].index("動詞")
            dst_morph = chunk.morphs[i_verb]
        except:
            continue

        src_morphs = []
        src_chunks: list[Chunk] = []
        for src in chunk.srcs:
            try:
                src_chunk = sentence[src]
                i_joshi = [morph.pos for morph in src_chunk.morphs][::-1].index("助詞")
                src_morph = src_chunk.morphs[-(i_joshi + 1)]
                src_morphs.append(src_morph)
                src_chunks.append(src_chunk)
            except:
                continue

        if len(src_morphs) == 0:
            continue

        print(f"{dst_morph.base}\t{' '.join(morph.base for morph in src_morphs)}\t{' '.join(chunk.to_str() for chunk in src_chunks)}")


用いる	を	道具を
する	て を	用いて 知能を
指す	を	一分野を
代わる	を に	知的行動を 人間に
行う	て に	代わって コンピューターに
する	も	研究分野とも
述べる	で は に	解説で 佐藤理史は 次のように
する	を で	知的能力を コンピュータ上で
する	を	推論判断を
する	を	画像データを
する	て を	解析して パターンを
ある	は が	応用例は 画像認識等が
する	に で により	1956年に ダートマス会議で ジョンマッカーシーにより
用いる	を	記号処理を
する	を と	記述を 主体と
使う	は でも	現在では 意味あいでも
呼ぶ	も	思考ルーチンも
ある	て も	使われている ことも
する	を	カウンセラーを
出す	が に	人工無脳が 引き合いに
する	に を	計算機に 役割を
呼ぶ	と	エキスパートシステムと
持つ	が に	人間が 暗黙に
なる	が と	記述が 問題と
する	が は が	出されるが 実現は 利用が
知る	は も	アプローチとしては アプローチも
ある	て が は に	困難視されている 知られているが 差は 記号的明示性に
集める	が を	サポートベクターマシンが 注目を
行う	を に を	経験を 元に 学習を
ある	も	手法も
する	を に	知性を 機械的に
する	において	宇宙において


### 47. 機能動詞構文のマイニング


In [9]:
for sentence in read_cabocha("./data/ch5-47.parsed"):
# for sentence in article:
    for chunk in sentence:
        try:
            i_verb = [morph.pos for morph in chunk.morphs].index("動詞")
            dst_morph = chunk.morphs[i_verb]
        except:
            continue

        src_morphs = []
        src_chunks: list[Chunk] = []
        for src in chunk.srcs:
            try:
                src_chunk = sentence[src]
                i_joshi = [morph.pos for morph in src_chunk.morphs][::-1].index("助詞")
                src_morph = src_chunk.morphs[-(i_joshi + 1)]
                src_morphs.append(src_morph)
                src_chunks.append(src_chunk)
            except:
                continue

        if len(src_morphs) == 0:
            continue

        chunk_sahen_and_wo = None
        i_sahen_and_wo = -1
        for i, src_chunk in enumerate(src_chunks):
            morphs = src_chunk.morphs
            if len(morphs) != 2:
                continue
            if not(morphs[0].pos == "名詞" and morphs[0].pos1 == "サ変接続"):
                continue
            if not(morphs[1].pos == "助詞" and morphs[1].base == "を"):
                continue
            chunk_sahen_and_wo = src_chunk
            i_sahen_and_wo = i

        if chunk_sahen_and_wo is None:
            continue

        src_chunks.pop(i_sahen_and_wo)
        src_morphs.pop(i_sahen_and_wo)

        print(f"{chunk_sahen_and_wo.to_str()}{dst_morph.base}\t{' '.join(morph.base for morph in src_morphs)}\t{' '.join(chunk.to_str() for chunk in src_chunks)}")


学習を行う	を に	経験を 元に


### 48. 名詞から根へのパスの抽出


In [10]:
for sentence in article[5:7]:
    for chunk in sentence:
        if chunk.dst is None:
            continue

        if "名詞" not in [morph.pos for morph in chunk.morphs]:
            continue

        cur_chunk = chunk
        while True:
            print(f"{cur_chunk.to_str()}", end="")
            if cur_chunk.dst is None:
                break
            print(" -> ", end="")
            cur_chunk = sentence[cur_chunk.dst]
        print()

2006年の -> ディープラーニング -> 登場と -> 登場により -> 行った -> なった
ディープラーニング -> 登場と -> 登場により -> 行った -> なった
深層学習の -> 登場と -> 登場により -> 行った -> なった
登場と -> 登場により -> 行った -> なった
2010年代 -> 以降の -> ビッグデータの -> 登場により -> 行った -> なった
以降の -> ビッグデータの -> 登場により -> 行った -> なった
ビッグデータの -> 登場により -> 行った -> なった
登場により -> 行った -> なった
一過性の -> 流行を -> 超えて -> 浸透して -> 行った -> なった
流行を -> 超えて -> 浸透して -> 行った -> なった
社会に -> 浸透して -> 行った -> なった
浸透して -> 行った -> なった
2016年から -> 2017年にかけて -> 導入した -> AIが -> 完全情報ゲームである -> 囲碁などの -> トップ棋士 -> プレイヤーも -> 破り -> なった
2017年にかけて -> 導入した -> AIが -> 完全情報ゲームである -> 囲碁などの -> トップ棋士 -> プレイヤーも -> 破り -> なった
ディープラーニングを -> 導入した -> AIが -> 完全情報ゲームである -> 囲碁などの -> トップ棋士 -> プレイヤーも -> 破り -> なった
導入した -> AIが -> 完全情報ゲームである -> 囲碁などの -> トップ棋士 -> プレイヤーも -> 破り -> なった
AIが -> 完全情報ゲームである -> 囲碁などの -> トップ棋士 -> プレイヤーも -> 破り -> なった
完全情報ゲームである -> 囲碁などの -> トップ棋士 -> プレイヤーも -> 破り -> なった
囲碁などの -> トップ棋士 -> プレイヤーも -> 破り -> なった
トップ棋士 -> プレイヤーも -> 破り -> なった
不完全情報ゲームである -> ポーカーの -> 世界トップクラスの -> プレイヤーも -> 破り -> なった
ポーカーの -> 世界トップクラスの -> プレイヤーも -> 破り ->

### 49. 名詞間の係り受けパスの抽出


In [11]:
from copy import deepcopy
import math

def masked_noun(chunk: Chunk, mask: str) -> str:
    morphs = deepcopy(chunk.morphs)
    morphs = [morph for morph in morphs if morph.pos != "記号"]
    i = 0
    # print([morph.pos for morph in morphs])
    while i < len(morphs)-1:
        if [morph.pos for morph in morphs[i:i+2]] == ["名詞", "名詞"]:
            morphs.pop(i)
        else:
            i += 1
    # print([morph.pos for morph in morphs])
    i_noun = [morph.pos for morph in morphs].index("名詞")
    # print([morph.pos for morph in morphs])
    text = "".join([morph.surface if i != i_noun else mask for i, morph in enumerate(morphs)])
    return text

for sentence in read_cabocha("./data/ch5_example.parsed"):
# for sentence in read_cabocha("./data/ch5-47.parsed"):
# for sentence in article[5:7]:

    # 係り受けの対応を全て取得
    kakariukes: list[list[int]] = []
    for i, chunk in enumerate(sentence):
        if chunk.dst is None:
            continue

        if "名詞" not in [morph.pos for morph in chunk.morphs]:
            continue

        cur_chunk = chunk
        path = [i]
        while True:
            if cur_chunk.dst is None:
                break
            path.append(cur_chunk.dst)
            cur_chunk = sentence[cur_chunk.dst]
        kakariukes.append(path)

    # print(kakariukes)
    for i in range(len(kakariukes)):
        for j in range(i + 1, len(kakariukes)):
            kakariukeA = kakariukes[i]
            kakariukeB = kakariukes[j]

            if kakariukeB[0] in kakariukeA:
                i_B = kakariukeA.index(kakariukeB[0])

                # Xを含む文節を表示
                i_A = 0
                cur_chunk = sentence[kakariukeA[i_A]]
                chunk_str = masked_noun(cur_chunk, mask="X")
                print(f"{chunk_str}", end="")
                i_A += 1

                # XからYまでの文節を表示
                while i_A != i_B:
                    print(" -> ", end="")
                    cur_chunk = sentence[kakariukeA[i_A]]
                    print(f"{cur_chunk.to_str()}", end="")
                    i_A += 1

                # Yを含む文節を表示
                print(" -> ", end="")
                cur_chunk = sentence[kakariukeA[i_B]]
                chunk_str = masked_noun(cur_chunk, mask="Y")
                print(f"{chunk_str}")
            else:
                intersection = None
                for i_A in range(1, len(kakariukeA)):
                    if not kakariukeA[i_A] in kakariukeB:
                        continue
                    i_B = kakariukeB.index(kakariukeA[i_A])
                    intersection = (i_A, i_B)
                    break

                if intersection is None:
                    continue

                dstA, dstB = intersection
                i_A = 0
                # Xを含む文節を表示
                cur_chunk = sentence[kakariukeA[i_A]]
                chunk_str = masked_noun(cur_chunk, mask="X")
                print(f"{chunk_str}", end="")
                i_A += 1

                # Xから共通地点までの文節を表示
                while i_A != dstA:
                    print(" -> ", end="")
                    cur_chunk = sentence[kakariukeA[i_A]]
                    print(f"{cur_chunk.to_str()}", end="")
                    i_A += 1

                print(" | ", end="")

                i_B = 0
                # Yを含む文節を表示
                cur_chunk = sentence[kakariukeB[i_B]]
                chunk_str = masked_noun(cur_chunk, mask="Y")
                print(f"{chunk_str}", end="")
                i_B += 1

                # Yから共通地点までの文節を表示
                while i_B != dstB:
                    print(" -> ", end="")
                    cur_chunk = sentence[kakariukeB[i_B]]
                    print(f"{cur_chunk.to_str()}", end="")
                    i_B += 1

                print(" | ", end="")

                # 共通地点の文節を表示
                cur_chunk = sentence[kakariukeB[dstB]]
                print(f"{cur_chunk.to_str()}")

Xは | Yに関する -> 最初の -> 会議で | 作り出した
Xは | Yの -> 会議で | 作り出した
Xは | Yで | 作り出した
Xは | Yという -> 用語を | 作り出した
Xは | Yを | 作り出した
Xに関する -> Yの
Xに関する -> 最初の -> Yで
Xに関する -> 最初の -> 会議で | Yという -> 用語を | 作り出した
Xに関する -> 最初の -> 会議で | Yを | 作り出した
Xの -> Yで
Xの -> 会議で | Yという -> 用語を | 作り出した
Xの -> 会議で | Yを | 作り出した
Xで | Yという -> 用語を | 作り出した
Xで | Yを | 作り出した
Xという -> Yを
